<a href="https://colab.research.google.com/github/pkumar2143/data_science_projects/blob/main/NYC_Airbnb_Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYC Airbnb Market Data Analysis**

A small data analysis project to obtain insights about NYC's Airbnb market.

Some tasks include finding out:
- What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.
- How many of the listings are private rooms? Save this into any variable.
- What is the average listing price? Round to the nearest two decimal places and save into a variable.
- Combine the new variables into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The DataFrame should only contain one row of values.

In [ ]:
# !pip install geopandas

In [1]:
import pandas as pd
import numpy as np

Airbnb Review Data

In [2]:
review_df = pd.read_csv('airbnb_last_review.tsv', sep='\t')
print(f"Review DF shape = {review_df.shape}")
review_df.head()

Review DF shape = (25209, 3)


,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


In [3]:
# Converting date-time string to a datetime stamp
review_df['last_review'] = pd.to_datetime(review_df['last_review'])
review_df.dtypes

,0
listing_id,int64
host_name,object
last_review,datetime64[ns]


The earliest and latest reviews are provided here:

In [4]:
earliest_review = min(review_df['last_review'])
latest_review   = max(review_df['last_review'])

print(f"Earliest review = {earliest_review} & Latest review = {latest_review}")

Earliest review = 2019-01-01 00:00:00 & Latest review = 2019-07-09 00:00:00


Airbnb Room Type Data

In [5]:
room_df = pd.read_excel('airbnb_room_type.xlsx')
print(f"Room DF shape = {room_df.shape}")
room_df.head()

Room DF shape = (25209, 3)


,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


The number of private room listings

In [6]:
room_df['room_type'].value_counts()

,count
room_type,
Entire home/apt,8458
Private room,7241
entire home/apt,2665
private room,2248
ENTIRE HOME/APT,2143
PRIVATE ROOM,1867
Shared room,380
shared room,110
SHARED ROOM,97


It seems there are multiple versions of the same category based on whether some parts of the category name are capitalized or not. Let's try to aggregate these.

I can implement two approaches:
- One specific to this dataset, where there are a reasonable number of room_type categories for me to manipulate.
- Generic in which the the number of categories may not be known, but really there are too many to manipulate manually; I will come back to this later. But the algorithm is:

Append category 1 (lower case): count to a dict.
for each category following:
  - check if category.lower() in the dict.
  - if so, add its respective value count to that category
  - if not, append the category:value to the dict.

In [7]:
# Specific method

room_type_counts = (room_df['room_type'].value_counts()).to_dict()
room_categories = {'entire home/apt':0, 'private room':0, 'shared room':0}

# for each value of room_type, check if it (after eliminating capitalization), whether it matches the room_category dict key
# if so, add its corresponding count to that category

for room_type in room_type_counts.keys():
  lowerc_rtype = room_type.lower()
  if lowerc_rtype in room_categories.keys():
    room_categories[lowerc_rtype] += room_type_counts[room_type]

print(f"room_categories dict = {room_categories}")

private_room_count = room_categories['private room']

print(f"Number of private rooms: {private_room_count}")

room_categories dict = {'entire home/apt': 13266, 'private room': 11356, 'shared room': 587}
Number of private rooms: 11356


So the total number of "Private room" listings is 11,356.

Airbnb Price Data

In [8]:
price_df = pd.read_csv('airbnb_price.csv')
print(f"Room DF shape = {price_df.shape}")
price_df.head()

Room DF shape = (25209, 3)


,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


The price column is a string containing the word "dollars", which we need to remove.

In [9]:
price_df['price'] = pd.to_numeric(price_df['price'].str.replace(' dollars',''))

In [10]:
price_desc = price_df['price'].describe()
print(f"Price summary = {price_desc}")

avg_price = float(round(price_desc['mean'], 2))

print(f"\n\nAvg price = {avg_price}")

Price summary = count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


Avg price = 141.78


The average price is $141.78

To combine what we learned into a summary DF

In [11]:
summary_df = pd.DataFrame([{'Earliest Review': earliest_review,
                           'Latest Review': latest_review,
                           'Nb Private Rooms': private_room_count,
                           'Avg Price': avg_price}])

summary_df

,Earliest Review,Latest Review,Nb Private Rooms,Avg Price
0,2019-01-01,2019-07-09,11356,141.78


We can obviously learn more about our dataset. For example,

- X -- Combine all data into one big df.
- X -- Find missing values.
- X -- Which listing has the latest/earliest review?
- X -- Which listing is the most/least expensive?
- X -- Which what are the summary statistics of each room_type?
- (Later... )Visualize the distribution of the listings on a map
  
  1. Can't be more specific than a NY neighborhood; perhaps a heatmap, showing which nhoods have the most listings?

In [12]:
# Find missing values

full_df = pd.merge(price_df, pd.merge(review_df, room_df, on='listing_id', how='outer'), on='listing_id', how='outer')
full_df.head()

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
0,2595,225,"Manhattan, Midtown",Jennifer,2019-05-21,Skylit Midtown Castle,Entire home/apt
1,3831,89,"Brooklyn, Clinton Hill",LisaRoxanne,2019-07-05,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,200,"Manhattan, Murray Hill",Chris,2019-06-22,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,79,"Manhattan, Hell's Kitchen",Shunichi,2019-06-24,Large Furnished Room Near B'way,private room
4,5238,150,"Manhattan, Chinatown",Ben,2019-06-09,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [13]:
full_df.isna().sum()

,0
listing_id,0
price,0
nbhood_full,0
host_name,8
last_review,0
description,10
room_type,0


Compared to the full count of 25,209 entries, only 8 hostnames and 10 descriptions are missing, so this is a pretty good dataset.

In [14]:
# Listing with the latest review
full_df[full_df['last_review'] == latest_review]

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
58,18152,200,"Manhattan, Upper East Side",Victoria,2019-07-09,Manhattan Room,Private room


In [15]:
# Listing(s) with the earliest review
full_df[full_df['last_review'] == earliest_review]

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
14,8025,80,"Brooklyn, Park Slope",Lisel,2019-01-01,CBG Helps Haiti Room#2.5,PRIVATE ROOM
98,31902,77,"Brooklyn, Flatlands",Sunder,2019-01-01,Sanctuary in East Flatbush,Private room
326,160609,200,"Manhattan, Upper West Side",Ligia,2019-01-01,LOCATION LOCATION LOCATION UWS 60's,Entire home/apt
335,165461,75,"Manhattan, Morningside Heights",Rahul,2019-01-01,Couldn't Be Closer To Columbia Uni2,Private room
444,242643,137,"Brooklyn, Williamsburg",Lauryn,2019-01-01,"BIG 1br, SLEEPS 4, dishwashr, TV",Entire home/apt
...,...,...,...,...,...,...,...
19667,30958853,32,"Brooklyn, Sunset Park",Fnu,2019-01-01,"Cozy room,25 minutes to Manhattan",Private room
19776,31066679,100,"Brooklyn, Bushwick",Dj,2019-01-01,Happy place to be,Private room
19815,31098262,250,"Manhattan, Chelsea",Kusum&Kannan,2019-01-01,1 Bedroom/1 Bath Apt in Hudson Yards/Midtown West,Entire home/apt
19836,31113241,200,"Queens, Flushing",Hasan,2019-01-01,"5 min walk to train,2 min walk to bus, most s...",Entire home/apt


Nearly 400 listings were last reviewed during the earliest review data (01-01-2019)

In [16]:
# Least expensive listing
full_df[full_df['price'] == min(full_df['price'])]

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
10861,20333471,0,"Bronx, East Morrisania",Anisha,2019-06-24,‚òÖHostel Style Room | Ideal Traveling Buddies‚òÖ,PRIVATE ROOM
10958,20523843,0,"Brooklyn, Bushwick",Martial Loft,2019-05-18,"MARTIAL LOFT 3: REDEMPTION (upstairs, 2nd room)",PRIVATE ROOM
11023,20639628,0,"Brooklyn, Bedford-Stuyvesant",Adeyemi,2019-06-15,Spacious comfortable master bedroom with nice ...,private room
11024,20639792,0,"Brooklyn, Bedford-Stuyvesant",Adeyemi,2019-06-21,Contemporary bedroom in brownstone with nice view,Private room
11025,20639914,0,"Brooklyn, Bedford-Stuyvesant",Adeyemi,2019-06-23,Cozy yet spacious private brownstone bedroom,PRIVATE ROOM
11577,21291569,0,"Brooklyn, Bushwick",Sergii,2019-06-22,Coliving in Brooklyn! Modern design / Shared room,Shared room
11589,21304320,0,"Brooklyn, Bushwick",Sergii,2019-05-24,Best Coliving space ever! Shared room.,Shared room


WOW, several free stays!!

In [17]:
# Most expensive listing
full_df[full_df['price'] == max(full_df['price'])]

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
23916,34895693,7500,"Brooklyn, East Flatbush",Sandra,2019-07-07,Gem of east Flatbush,Private room


In [18]:
# Which what are the summary statistics of each room_type?
# for reference : room_categories = {'entire home/apt':0, 'private room':0, 'shared room':0}

# convert room_type catergories to lower case for ease
full_df['room_type'] = full_df['room_type'].str.lower()

pd.set_option('display.precision', 2)

for room_cat in room_categories.keys():
  print(f"Price summary for {room_cat} = \n\n {full_df[full_df['room_type'] == room_cat].describe().drop(['listing_id','last_review'], axis=1)} \n\n")

Price summary for entire home/apt = 

           price
count  13266.00
mean     197.17
min       10.00
25%      120.00
50%      159.00
75%      225.00
max     5100.00
std      165.80 


Price summary for private room = 

           price
count  11356.00
mean      81.64
min        0.00
25%       51.00
50%       70.00
75%       94.00
max     7500.00
std       91.50 


Price summary for shared room = 

         price
count  587.00
mean    53.47
min      0.00
25%     30.00
50%     41.00
75%     65.00
max    800.00
std     45.69 




Obviously, the average price for an entire home/apt is nearly double than for a private room, which in turn is about 50% more than for a shared room.

Also interestingly, the most expensive private room is about $2000 more than the most expensive full home/apt.

#### Geographic Data

In [25]:
import geopandas as gpd

In [21]:
# Extracting neighborhood statistics

full_df['nbhood_full'].value_counts()

,count
nbhood_full,
"Brooklyn, Bedford-Stuyvesant",2209
"Brooklyn, Williamsburg",1853
"Manhattan, Harlem",1435
"Brooklyn, Bushwick",1202
"Manhattan, Hell's Kitchen",1119
...,...
"Staten Island, Rossville",1
"Staten Island, Willowbrook",1
"Bronx, West Farms",1


In [27]:
nyc_map = gpd.read_file("MODZCTA_2010.shp")

In [28]:
nyc_map

,MODZCTA,label,geometry
0,10001,"10001, 10118","POLYGON ((987646 210360, 987522 210136, 987296..."
1,10002,10002,"POLYGON ((985342 197658, 985336 197688, 985318..."
2,10003,10003,"POLYGON ((987412 202684, 987400 202660, 987366..."
3,10004,10004,"MULTIPOLYGON (((982030 197212, 981958 197118, ..."
4,10005,10005,"POLYGON ((982126 195468, 982080 195430, 982072..."
...,...,...,...
173,11692,11692,"POLYGON ((1038916 153042, 1038686 153014, 1038..."
174,11693,11693,"MULTIPOLYGON (((1034996 151848, 1034886 151806..."
175,11694,11694,"POLYGON ((1022824 146064, 1022582 145920, 1022..."
176,11697,11697,"POLYGON ((1012278 142148, 1012284 142124, 1012..."


In [29]:
# Convert airbnb nhoods to MODZCTA or capture their geometric centers